In [2]:
import pandas as pd
import numpy as np
import feather
import lightgbm

In [3]:
params = {
    'application':'binary',
    'num_leaves': 31,
    'max_depth': 20,
    'feature_fraction': 0.7,
    'bagging_fraction': 0.7,
    'max_bin': 200,
    'metric': 'auc',
    'verbose': 1
    
}
cols = ['genre_id', 'media_id', 'album_id', 'context_type',
       'release_date', 'platform_name', 'platform_family', 'media_duration',
       'listen_type', 'user_gender', 'user_id', 'artist_id', 'user_age']

In [5]:
for i in range(1,4):
    train = feather.read_dataframe("folds/train_{}.feather".format(i))
    test = feather.read_dataframe("folds/test_{}.feather".format(i))
    lgb_train = lightgbm.Dataset(train[cols], train["is_listened"])
    lgb_test = lightgbm.Dataset(test[cols], test["is_listened"])
    evals = {}
    model_lgm = lightgbm.train(params, lgb_train, 1000, valid_sets=lgb_test, verbose_eval=1, evals_result=evals)
    print(evals["valid_0"]["auc"][-1])

[1]	valid_0's auc: 0.636673
[2]	valid_0's auc: 0.635543
[3]	valid_0's auc: 0.636141
[4]	valid_0's auc: 0.637768
[5]	valid_0's auc: 0.637529
[6]	valid_0's auc: 0.639198
[7]	valid_0's auc: 0.638531
[8]	valid_0's auc: 0.63756
[9]	valid_0's auc: 0.637556
[10]	valid_0's auc: 0.63842
[11]	valid_0's auc: 0.638681
[12]	valid_0's auc: 0.639659
[13]	valid_0's auc: 0.639968
[14]	valid_0's auc: 0.64052
[15]	valid_0's auc: 0.64034
[16]	valid_0's auc: 0.640405
[17]	valid_0's auc: 0.641044
[18]	valid_0's auc: 0.641412
[19]	valid_0's auc: 0.641339
[20]	valid_0's auc: 0.641834
[21]	valid_0's auc: 0.642243
[22]	valid_0's auc: 0.643011
[23]	valid_0's auc: 0.643552
[24]	valid_0's auc: 0.644015
[25]	valid_0's auc: 0.644426
[26]	valid_0's auc: 0.644945
[27]	valid_0's auc: 0.645495
[28]	valid_0's auc: 0.645713
[29]	valid_0's auc: 0.646393
[30]	valid_0's auc: 0.64673
[31]	valid_0's auc: 0.647017
[32]	valid_0's auc: 0.647479
[33]	valid_0's auc: 0.647802
[34]	valid_0's auc: 0.64798
[35]	valid_0's auc: 0.648205


In [7]:
(0.659522413457 + 0.650899076427 + 0.656177775181) / 3

0.6555330883549999

In [12]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")
train = train[train["listen_type"] == 1]

In [13]:
lgb_train = lightgbm.Dataset(train[cols], train["is_listened"])
model_lgm = lightgbm.train(params, lgb_train, 1000)

In [14]:
lgbm_preds = model_lgm.predict(test[cols])

In [15]:
submition = pd.read_csv("../data/sample_submission_kaggle.csv")

In [16]:
lgbm_preds

array([ 0.65843271,  0.67511827,  0.69296434, ...,  0.80782184,
        0.59795821,  0.76414823])

In [17]:
submition["is_listened"] = lgbm_preds

In [18]:
submition.to_csv("submit_lgm_flow.csv", index=False)

In [20]:
model_lgm.feature_importance()

array([1854, 1751, 1381, 2910, 2289, 1449,  994, 2159,    0, 1260, 7342,
       2694, 3917])

In [21]:
cols

['genre_id',
 'media_id',
 'album_id',
 'context_type',
 'release_date',
 'platform_name',
 'platform_family',
 'media_duration',
 'listen_type',
 'user_gender',
 'user_id',
 'artist_id',
 'user_age']

In [23]:
train[cols].dtypes

genre_id           int64
media_id           int64
album_id           int64
context_type       int64
release_date       int64
platform_name      int64
platform_family    int64
media_duration     int64
listen_type        int64
user_gender        int64
user_id            int64
artist_id          int64
user_age           int64
dtype: object